In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import io
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon
import os

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.animation import FuncAnimation
import glob
import random

from scipy.optimize import curve_fit

from astropy.io import fits
from astropy.time import Time
from astropy import units as u
import scipy.integrate as integrate
import lightkurve

from matplotlib.path import Path
import matplotlib.patches as patches
%matplotlib inline

import math
import datetime as dt
import lightkurve.io as l
from lightkurve.io.generic import read_generic_lightcurve

In [2]:
def clean_text(str):
    s = result = ''.join([i for i in archivo[20:] if not i.isdigit()])
    t = s.replace('_curves.txt', '')
    u = t.replace('+', '')
    u = u.replace('-', '')
    if 'KsJs' in u:
        u = u.replace('KsJs', 'Ks Js')
    if u[0] == '_':
        u = u[1:]
    if 'AB' in u:
        u = u.replace('AB', '')
    return u

In [3]:
# Function is created in order to eliminate characters which are not Printable ASCII characters that appear 
# from the data transfer from repository
def textcleaner(str1):
    w = str(str1).split('\n')[0]
    x = str(w)
    y = str(x.split('  ')[2])
    z = y.replace("'", "")
    return z

In [4]:
# Function is created in order to eliminate characters which are not Printable ASCII characters that appear 
# from the data transfer from repository
def numcleaner(str1):
    w = str(str1).split('\n')[0]
    x = str(w)
    x.split('  ')
    if ' NaN' in x:
        y = str(x.split('  ')[1])
        z = y.replace("'", "")
        a = z.replace(" ", "")
        return a
    else:
        w = str(str1).split('\n')[0]
        x = str(w)
        y = str(x.split('  ')[2])
        z = y.replace("'", "")
        return z

In [5]:
#repository like the one we have online
general_data = pd.read_csv('/Users/kmanj/FITS_BD_Files/Repository_Data/Repository.csv', encoding ='latin1')

In [6]:
dir = '/Users/kmanj/Manjavacas Research/Metchev_2015/Metchev2015/Non-Variables/'
dir_save = '/Users/kmanj/FITS_BD_Files/Data/Metchev_2015/NV/'

In [7]:
dwarfs_files = os.listdir(dir)
#seperating the ch1 and ch2
dwarf_dat_ch1 = [archive for archive in dwarfs_files if archive.endswith("1.txt")]
dwarf_dat_ch2 = [archive for archive in dwarfs_files if archive.endswith("ch2.txt")]
dwarf_dat_ch1;
dwarf_dat_ch2;

In [8]:
new_name_file_ch1 = []
new_name_ch1 = []

# Iterate over the elements of the first list.
for archivo in dwarf_dat_ch1:
    if archivo.startswith('2M'):
        # Extract the first 5 characters of the file name if 2MASS object
        first_5_characteres = 'J' + str(archivo[2:6])
    elif archivo.startswith('ROSS'):
        first_5_characteres = 'J1300'
    elif archivo.startswith('SDSS'):
        first_5_characteres = 'J' + str(archivo[4:8])
    elif archivo.startswith('ULAS'):
        first_5_characteres = 'J1416'
    
    # Iterate over the elements of the second list.
    for valor in general_data['Name_file']:
        # Extract the first 5 characters of the reference value
        valor = str(valor)
        first_5_values = valor[:5]
        
        # Compare the first 5 characters of the two elements
        if first_5_characteres == first_5_values:
            # Modify the file name to the desired format
            nuevo_nombre = valor + ".dat"
            # Add the new name to the new list
            new_name_ch1.append(valor)
            new_name_file_ch1.append(nuevo_nombre)
            # Break the inner loop once a match is found.
            break

new_name_file_ch1;
new_name_ch1;

new_name_file_ch2 = []
new_name_ch2 = []

# Iterate over the elements of the first list.
for archivo in dwarf_dat_ch2:
    if archivo.startswith('2M'):
        # Extract the first 5 characters of the file name if 2MASS object
        first_5_characteres = 'J' + str(archivo[2:6])
    elif archivo.startswith('ROSS'):
        first_5_characteres = 'J1300'
    elif archivo.startswith('SDSS'):
        first_5_characteres = 'J' + str(archivo[4:8])
    elif archivo.startswith('ULAS'):
        first_5_characteres = 'J1416'
    
    # Iterate over the elements of the second list.
    for valor in general_data['Name_file']:
        # Extract the first 5 characters of the reference value
        valor = str(valor)
        first_5_values = valor[:5]
        
        # Compare the first 5 characters of the two elements
        if first_5_characteres == first_5_values:
            # Modify the file name to the desired format
            nuevo_nombre = valor + ".dat"
            # Add the new name to the new list
            new_name_ch2.append(valor)
            new_name_file_ch2.append(nuevo_nombre)
            # Break the inner loop once a match is found.
            break

new_name_file_ch2;
new_name_ch2;

### Channel 1

In [9]:
dwarfs_data = []
for i in range(len(new_name_file_ch1)):
    data = pd.read_csv("{}{}".format(dir,dwarf_dat_ch1[i]), header=None, sep='\t')
    dwarfs_data.append(data)

In [31]:
dwarfs_data;

In [32]:
Reference = 'Metchev et al 2015 ApJ 799 154'
Instrument = 'Infrared Array Camera'
Date_obs = 'Published January 27, 2015'
Band_obs = 'Ch1'

In [33]:
Object = []
SpectralType = []
Jmag = []
Hmag = []
Kmag = []
RA = []
Dec = []
dJmag = []
dHmag = []
dKmag = []


for i in range(len(new_name_file_ch1)): 
    Object.append(textcleaner(general_data['Name'][general_data['Name_file'] == new_name_ch1[i]]))
    SpectralType.append(textcleaner(general_data['SpT '][general_data['Name_file'] == new_name_ch1[i]]))
    Jmag.append(textcleaner(general_data['J'][general_data['Name_file'] == new_name_ch1[i]]))
    dJmag.append(numcleaner(general_data['dJ'][general_data['Name_file'] == new_name_ch1[i]]))
    Hmag.append(numcleaner(general_data['H'][general_data['Name_file'] == new_name_ch1[i]]))
    dHmag.append(numcleaner(general_data['dH'][general_data['Name_file'] == new_name_ch1[i]]))
    Kmag.append(numcleaner(general_data['K'][general_data['Name_file'] == new_name_ch1[i]]))
    dKmag.append(numcleaner(general_data['dK'][general_data['Name_file'] == new_name_ch1[i]]))
    RA.append(textcleaner(general_data['RA'][general_data['Name_file'] == new_name_ch1[i]]))
    Dec.append(textcleaner(general_data['DEC'][general_data['Name_file'] == new_name_ch1[i]]))

In [42]:
Object1

['2MASSI J0328426+230205',
 '2MASS J04210718-6306022',
 '2MASS J05160945-0445499',
 '2MASSW J0820299+450031',
 '2MASS J09490860-1545485',
 '2MASS J10595185+3042059',
 '2MASS J11220826-3512363',
 '2MASS J12095613-1004008',
 '2MASSI J1726000+153819',
 '2MASSW J2224438-015852',
 '2MASSI J2254188+312349',
 'Ross 458c',
 'SDSS J085834.42+325627.7',
 'SDSS J115013.17+052012.3',
 'SDSS J125453.90-012247.5',
 'SDSS J152039.82+354619.8',
 'SDSS J154508.93+355527.3',
 'SDSS J204317.69-155103.4',
 'SDSS J205235.31-160929.8',
 'SDSSp J224953.45+004404.2']

In [16]:
RA = [
    52.177709,
    65.279958,
    79.039376,
    125.124834,
    147.285834,
    164.96604,
    170.5344,
    182.484,
    261.50029,
    336.182541,
    343.5788,
    195.173925,
    134.643,
    177.555,
    193.724709,
    230.16558,
    236.2872,
    310.8237,
    313.1465,
    342.472709
]

Dec = [
    23.03475,
    -63.100625,
    -4.763861,
    45.00875,
    -15.763472,
    30.701639,
    -35.2102,
    -10.0671,
    15.638611,
    -1.981139,
    31.3972,
    12.354089,
    32.9407,
    5.3369,
    -1.379833,
    35.7725,
    35.9245,
    -15.8509,
    -16.1585,
    0.734611
]
len(RA)

20

In [14]:
name_files = []
for i in range(len(dwarfs_data)):
    lc = l.LightCurve(time = list(dwarfs_data[i][0]), flux = list(dwarfs_data[i][1]) * u.dimensionless_unscaled)
    name = dir_save + Object[i]
    contador = 1
    while name + '_Ch1_Metc2015.fits' in name_files:
        name = name + '_' + str(contador)
        contador += 1
    else:
        lc.to_fits(path = name + '_Ch1_Metc2015.fits')
        name_files.append(name + '_Ch1_Metc2015.fits')

In [19]:
for i in range(len(dwarfs_data)):
    with fits.open(name_files[i], mode = 'update') as f:
        f[0].header['OBJECT'] = Object[i]
        f[0].header['SPECTYPE'] = SpectralType[i]
        f[0].header['TIMESYS'] = 'mjd'
        f[0].header['TELESCOP'] = 'Spitzer Space Telescope'
        f[0].header['INSTRUME'] = Instrument
        f[0].header['FILTER'] = Band_obs
        f[0].header['VOREF'] = Reference
        f[0].header['RA_OBJ'] = RA[i]
        f[0].header['DEC_OBJ'] = Dec[i]
        f[0].header['JMAG'] = Jmag[i] + '+/-' + dJmag[i]
        f[0].header['HMAG'] = Hmag[i] + '+/-' + dHmag[i]
        f[0].header['KMAG'] = Kmag[i] + '+/-' + dKmag[i]
        f[1].header['TUNIT1'] = 'Hrs'

### Channel 2

In [43]:
dwarfs_data = []
for i in range(len(new_name_file_ch2)):
    data = pd.read_csv("{}{}".format(dir,dwarf_dat_ch2[i]), header=None, sep='\t')
    dwarfs_data.append(data)

In [44]:
Reference = 'Published in The Astrophysical Journal, Volume 799, Number 2: Metchev et. al 2015'
Instrument = 'Infrared Array Camera '
Date_obs = 'Published January 27, 2015'
Band_obs = 'Ch2'
Telescope = 'Spitzer Space Telescope'

In [45]:
Object = []
SpectralType = []
Jmag = []
Hmag = []
Kmag = []
RA = []
Dec = []
dJmag = []
dHmag = []
dKmag = []


for i in range(len(new_name_file_ch2)): 
    Object.append(textcleaner(general_data['Name'][general_data['Name_file'] == new_name_ch2[i]]))
    SpectralType.append(textcleaner(general_data['SpT '][general_data['Name_file'] == new_name_ch2[i]]))
    Jmag.append(textcleaner(general_data['J'][general_data['Name_file'] == new_name_ch2[i]]))
    dJmag.append(numcleaner(general_data['dJ'][general_data['Name_file'] == new_name_ch2[i]]))
    Hmag.append(numcleaner(general_data['H'][general_data['Name_file'] == new_name_ch2[i]]))
    dHmag.append(numcleaner(general_data['dH'][general_data['Name_file'] == new_name_ch2[i]]))
    Kmag.append(numcleaner(general_data['K'][general_data['Name_file'] == new_name_ch2[i]]))
    dKmag.append(numcleaner(general_data['dK'][general_data['Name_file'] == new_name_ch2[i]]))
    RA.append(textcleaner(general_data['RA'][general_data['Name_file'] == new_name_ch2[i]]))
    Dec.append(textcleaner(general_data['DEC'][general_data['Name_file'] == new_name_ch2[i]]))

In [46]:
RA = [
    52.177709,
    65.279958,
    79.039376,
    125.124834,
    147.285834,
    164.96604,
    170.5344,
    182.484,
    261.50029,
    336.182541,
    343.5788,
    195.173925,
    27.923126,
    134.643,
    177.555,
    193.724709,
    230.16558,
    236.2872,
    310.8237,
    313.1465,
    342.472709
]

Dec = [
    23.03475,
    -63.100625,
    -4.763861,
    45.00875,
    -15.763472,
    30.701639,
    -35.2102,
    -10.0671,
    15.638611,
    -1.981139,
    31.3972,
    12.354089,
    12.741667,
    32.9407,
    5.3369,
    -1.379833,
    35.7725,
    35.9245,
    -15.8509,
    -16.1585,
    0.734611
]
len(RA)

21

In [47]:
Object

['2MASSI J0328426+230205',
 '2MASS J04210718-6306022',
 '2MASS J05160945-0445499',
 '2MASSW J0820299+450031',
 '2MASS J09490860-1545485',
 '2MASS J10595185+3042059',
 '2MASS J11220826-3512363',
 '2MASS J12095613-1004008',
 '2MASSI J1726000+153819',
 '2MASSW J2224438-015852',
 '2MASSI J2254188+312349',
 'Ross 458c',
 'SDSS J015141.69+124429.6',
 'SDSS J085834.42+325627.7',
 'SDSS J115013.17+052012.3',
 'SDSS J125453.90-012247.5',
 'SDSS J152039.82+354619.8',
 'SDSS J154508.93+355527.3',
 'SDSS J204317.69-155103.4',
 'SDSS J205235.31-160929.8',
 'SDSSp J224953.45+004404.2']

In [48]:
len(Object2)

21

In [50]:
name_files = []
for i in range(len(dwarfs_data)):
    lc = l.LightCurve(time = list(dwarfs_data[i][0]), flux = list(dwarfs_data[i][1]) * u.dimensionless_unscaled)
    name = dir_save + Object[i]
    contador = 1
    while name + '_Ch2_Metc2015.fits' in name_files:
        name = name + '_' + str(contador)
        contador += 1
    else:
        lc.to_fits(path = name + '_Ch2_Metc2015.fits')
        name_files.append(name + '_Ch2_Metc2015.fits')

In [51]:
for i in range(len(dwarfs_data)):
    with fits.open(name_files[i], mode = 'update') as f:
        f[0].header['OBJECT'] = Object[i]
        f[0].header['SPECTYPE'] = SpectralType[i]
        f[0].header['TIMESYS'] = 'mjd'
        f[0].header['TELESCOP'] = 'Spitzer Space Telescope'
        f[0].header['INSTRUME'] = Instrument
        f[0].header['FILTER'] = Band_obs
        f[0].header['VOREF'] = Reference
        f[0].header['RA_OBJ'] = RA[i]
        f[0].header['DEC_OBJ'] = Dec[i]
        f[0].header['JMAG'] = Jmag[i] + '+/-' + dJmag[i]
        f[0].header['HMAG'] = Hmag[i] + '+/-' + dHmag[i]
        f[0].header['KMAG'] = Kmag[i] + '+/-' + dKmag[i]
        f[1].header['TUNIT1'] = 'Hrs'